# Introductie `xarray`

Deze notebook bevat een korte introductie van `xarray` de package waarmee we de data die we in `nlmod` bewerken wordt opgeslagen en beheerd.

De volgende onderwerpen komen aan bod:

- de twee basis data structuren in xarray: `Dataset` en `DataArray`
- hoe je interacteert met of bewerkingen doet op die data structuren
- visualisatie van de data
- wegschrijven en inlezen van data

Aan het einde van het notebook is een kleine opdracht opgenomen.

De eerste stap is het importeren van de packages. We importeren hier ook nlmod voor het ophalen van onze voorbeeld dataset.

In [ ]:
import nlmod
import xarray as xr
import numpy as np

Als voorbeeld gebruiken we REGIS data die we binnen een stukje van Nederland ophalen. De functie `get_regis()` van `nlmod` haalt REGIS op als `xarray.Dataset`.

In [ ]:
# voorbeeld dataset is REGIS binnen een stukje van NL
extent = [118_000, 122_000, 473_000, 476_000]
ds = nlmod.read.get_regis(extent)

Waternet optie:

In [ ]:
ds = xr.open_dataset("regis.nc")

## `xarray` data structuren

Een `Dataset` is een soort dictionary met daarin zogenaamde `DataArrays` die ieder een eigen label (key) hebben. `Datasets` hebben een html-weergave waardoor we de inhoud in een notebook handig kunnen bekijken:

In [ ]:
ds

We kunnen de `top` `DataArray` bekijken door deze uit de `Dataset` op te vragen net als in een dictionary:

In [ ]:
ds["top"]

Een `DataArray` is eigenlijk een array met gelabelde dimensies en metadata. De eerste dimensie van de `top` is dus het laagnummer en de volgende twee dimensies zijn de `y` en `x` data.

De metadata staat opgeslagen onder de attributes, en daar staat bijvoorbeeld in dat de eenheid van de data meters NAP is.

In [ ]:
top = ds['top']
top.dims  # labels van de dimensies

In [ ]:
# metadata, opgeslagen onder .attrs
top.attrs["units"]

De ruimtelijke informatie (in dit geval) over de dimensies wordt opgeslagen onder `.coords`

In [ ]:
top.coords

Het ophalen van de data werkt eigenlijk het zelfde als voor de data variabelen zoals "top":

In [ ]:
top["x"]

Elke `DataArray` is eigenlijk een numpy.array met daaroverheen extra data. Om de numpy array op te halen gebruik je `.data`

In [ ]:
top.data

## Indexing (selecteren) van data 

Maar wat kunnen we nu met deze `xarray` data structuren? Een handige toepassing is het selecteren van data op basis van een label of op basis van een index.

Het selecteren van data op basis van een index doe je met `.isel()`. Om de top van de bovenste laag te selecteren:

In [ ]:
top.isel(layer=0)

Maar dat kan ook op basis van de naam de laag. Dat heet label-based indexing en doe je met `.sel()`:

In [ ]:
top.sel(layer='HLc')

Als je meerdere lagen tegelijk wilt selecteren kan dat met `slice()`:

In [ ]:
# selecteren van Holoceen tot en met 3de zandlaag van formatie van Kreftenheye
top.sel(layer=slice("HLc", "KRz3"))

We kunnen ook de laagindeling op een specifieke plek opvragen:

In [ ]:
top.sel(x=118_350, y=474_850) # .to_pandas()

En als we gewoon een coordinaat willen intypen en de dichtstbijzijnde informatie willen opvragen gebruiken we `method="nearest"`:

In [ ]:
top.sel(x=118_412, y=474_911, method="nearest")

## Visualisatie

`xarray` bevat ook functionaliteit voor het visualiseren van data:

In [ ]:
top.sel(layer="HLc").plot();

Als we eens doorsnede selecteren en plotten kunnen we zien waar welke laag voorkomt:

In [ ]:
top.sel(y=474_911, method='nearest').plot();

Of we kunnen de bovenkant van de lagen als lijn plotten:

In [ ]:
top.sel(y=474_911, method='nearest').plot.line(x="x");

We kunnen ook het bovenaanzicht van meerdere lagen tegelijk plotten:

In [ ]:
top.sel(layer=slice("HLc", "KRz3")).plot(col="layer", col_wrap=2)

## Berekeningen

`xarray.DataArrays` werken verder heel vergelijkbaar als `numpy.arrays`. Je kan dus numpy functies toepassen op xarray objecten, maar xarray bevat zelf ook functies om berekeningen uit te voeren.

Bijvoorbeeld het gemiddelde niveau uitrekenen van de bovenste modellaag:

In [ ]:
top.sel(layer="HLc").mean()

In [ ]:
# hier komt hetzelfde uit
np.nanmean(top.sel(layer="HLc"))

Het handige aan xarray functies is dat je de naam van de dimensies waarover een operatie wil uitvoeren kan opgeven. Zo kan je dus bijvoorbeeld het gemiddelde diepte van de bovenkant van elke laag berekenen op deze manier:

In [ ]:
top.mean(dim=("y", "x"))

Ook simpele sommetjes kunnen we uitvoeren met deze data en opslaan in een nieuwe variabele. Bijvoorbeeld het verschil tussen de bovenkant van de eerste en de tweede laag.

In [ ]:
thickness = top.isel(layer=0) - ds["botm"].isel(layer=0)
thickness

## Opslaan en inlezen van data

`xarray` data kan je eenvoudig opslaan in NetCDF formaat met de functie `ds.to_netcdf()`

In [ ]:
ds.to_netcdf("regis_dit_is_nieuw.nc")

En deze dataset kunnen we ook weer inlezen met `xr.open_dataset()`

In [ ]:
ds2 = xr.open_dataset("regis_dit_is_nieuw.nc")

## Opdracht

Voer de volgende stappen uit, elke in een eigen code cel:

1. Bereken de dikte van alle lagen.
2. Bereken de gemiddelde dikte van alle lagen.
3. Selecteer de data in het gebied tussen xmin, xmax = (118,050, 120,050) en ymin, ymax = (473_050, 474_950). (Let hierbij op dat de y-data aflopend is, van hoog naar laag!)
4. Plot de dikte van de bovenste 4 lagen in bovenaanzicht.
5. Sla de dikte, berekend in stap 1, op als NetCDF bestand.
6. Lees dit bestand weer in en laat zien dat de het verschil in dikte van de bovenste laag tussen het ingelezen bestand en die uit stap 1 overal 0.0 is. 